# Chapter-3: Create and train a ML Segmentation Model using AWS Sagemaker

The objectives you complete during the course of this chapter introduce you to the process of implementing the SageMaker model training tool. You will engage in this process by completing the following objectives:

### AWS SageMaker:
Amazon SageMaker helps data scientists and developers to prepare, build, train, and deploy high-quality machine learning (ML) models quickly by bringing together a broad set of capabilities purpose-built for ML.

Simply put, It is a set of cloud based (specifically, AWS) apps that focus on labeling, training, testing and deploying models.

## How it Works in context of our HLD problem:

### Prepare:

Sagemaker provides in-house labelling tools and data wrangling tools for some common ML workflows. Owing to the spatio temporal nature of the HLD dataset, we will be skipping this functionality of sagemaker and use IMPACT-Built ImageLabeler tool for identifying features and labeling them. This workflow has been covered in Chapter-0 and Chapter-1 of this workshop

### Build, Train & Tune:

Sagemaker provides access to cloud-hosted jupyter notebooks along with pre-built ML models. For our purposes, The model we are using for this demo is Unet segmentation model (https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/). The architecture is a stack of convolutions followed by de-convolutions. This model assigns a class label to each pixel of the input and gives a output matching the size of the input. The resulting output, once trained with high-latitude dust(HLD) masks, will segment any given image to HLD and non-HLD pixels. we will be covering the process in this chapter (chapter-3).

### Deploy and Manage:

Sagemaker provides endpoints to infer from the trained models. This functionality will be showcased in Chapter-4


In [ ]:
!pip install -r src/requirements.txt

## Required Imports

In [ ]:
import boto3
import fiona

import math
import numpy as np
import os
import random
import rasterio.features
import re
import requests
import shutil

from datetime import datetime
from glob import glob
from io import BytesIO
from IPython.display import Image as Display
from PIL import Image


## From Chapter-2: setup access, download, and process data into ML ready format.

In [ ]:
ACCOUNT_NUMBER = "350996086543"
ROLE_NAME = "notebookAccessRole"
ROLE_ARN = f"arn:aws:iam::{ACCOUNT_NUMBER}:role/{ROLE_NAME}"
SOURCE_BUCKET = "impact-datashare"
DESTINATION_BUCKET = f"s3://{ACCOUNT_NUMBER}-model-bucket"

# NOTE: Use image_url function above to create a valid url, if the shapefile generation was not done in Aqua, TrueColor 
DATA_FOLDER = "data"
EVENT = "hld-labeled"
IMAGE_FOLDER = "images"
SHAPEFILE_FOLDER = "shapefiles"
LOG_FOLDER = "logs"
URL = "https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?SERVICE=WMS&REQUEST=GetMap&layers=MODIS_Aqua_CorrectedReflectance_TrueColor&version=1.3.0&crs=EPSG:4326&transparent=false&width={}&height={}&bbox={}&format=image/tiff&time={}"
KM_PER_DEG_AT_EQ = 111.
RESOLUTION = 0.25

def assumed_role_session():
    # Assume the "notebookAccessRole" role we created using AWS CDK.
    client = boto3.client('sts')
    creds = client.assume_role(
        RoleArn=ROLE_ARN,
        RoleSessionName=ROLE_NAME
    )['Credentials']
    return boto3.session.Session(
        aws_access_key_id=creds['AccessKeyId'],
        aws_secret_access_key=creds['SecretAccessKey'],
        aws_session_token=creds['SessionToken'],
        region_name='us-east-1'
    )

def mkdir(foldername):
    if os.path.exists(foldername):
        print(f"'{foldername}' folder already exists.")
        return
    os.makedirs(foldername)
    print(f"Created folder: {foldername}")

    
def delete_folder(foldername):
    shutil.rmtree(foldername) 
    
# Remove already existing folder for a split and create a new one with passed filenames
def create_split(split, files):
    """
    Clear and create folder with new files.
    split: choice of "train", "test", and "val"
    files: list of tiff file paths
    """
    print(f'Preparing {split} split with {len(files)} examples.')
    folder_name = f"{DATA_FOLDER}/{split}"
    if os.path.exists(folder_name):
        delete_folder(folder_name)
    mkdir(folder_name)
    for filename in files:
        internal_filename = filename.split('/')[-1]
        bitmap_filename = filename.replace('.tiff', '_bitmap.png')
        shutil.copyfile(filename, f"{folder_name}/{internal_filename}")
        shutil.copyfile(bitmap_filename, f"{folder_name}/{bitmap_filename.split('/')[-1]}")
    
def prepare_splits(source_folder, splits={'train': 0.6, 'val': 0.2, 'test': 0.2}):
    files = glob(f"{source_folder}/*.tiff")
    print(f"Total examples found: {len(files)}")
    random.shuffle(files)
    length = len(files)
    train_limit = math.ceil(length * splits['train'])
    val_limit = train_limit + math.ceil(length * splits['train'])
    create_split('train', files[0:train_limit])
    create_split('val', files[train_limit:val_limit])
    create_split('test', files[train_limit:val_limit])


In [ ]:
session = assumed_role_session()  # Create a aws session with appropriate permissions
prepare_datasets(session) # prepare ML Ready data from shapefiles labeled in Imagealabeler 
download_dataset(session) # Download preprocessed data (just in case prepare_dataset() doesn't work)
prepare_splits(f"{DATA_FOLDER}/{IMAGE_FOLDER}") # split data into train/val/test chunks
!cp data ../ # move data to parent folder

## Initialize a sagemaker session to upload data

In [ ]:
import sagemaker
sagemaker_session = sagemaker.Session()
train_images = sagemaker_session.upload_data(path='../data/train')
val_images = sagemaker_session.upload_data(path='../data/val')
test_images = sagemaker_session.upload_data(path='../data/test')

## Attach a Tensorboard session

In [ ]:
aws_region = sagemaker_session.boto_region_name
tensorflow_logs_path = f'{DESTINAION_BUCKET}/{DATA_FOLDER}/{LOG_FOLDER}'
!AWS_REGION={aws_region} tensorboard --logdir {tensorflow_logs_path}

## Use tensorflow wrapper provided by sagemaker to train a unet model

The SageMaker Python SDK TensorFlow estimators and models and the SageMaker open-source TensorFlow containers make writing a TensorFlow script and running it in SageMaker easier.

We define a sagemaker instance using `sagemaker.tensorflow.Tensorflow` class. 

- `entry_point` parameter should point to the underlying tensorflow model implementation.
- `source_dir` points to the folder that contains the `entry_point`.
- `role` should be given tthe appropriate notebook access role we've created in chaper-2
- `instance_count`, `instance_type` defines the number of instances and the type of instance of the EC2 instance that will be used for compute.
- `output_path` - dictates where the output files from training the model will reside
- `image_uri` should point t the appropriate tensorflow ecr container image


In [ ]:
from sagemaker.tensorflow import TensorFlow
keras_metric_definition = [
    {"Name": "train:loss", "Regex": ".*loss: ([0-9\\.]+) - accuracy: [0-9\\.]+.*"},
    {"Name": "train:accuracy", "Regex": ".*loss: [0-9\\.]+ - accuracy: ([0-9\\.]+).*"},
    {
        "Name": "validation:accuracy",
        "Regex": ".*step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_accuracy: ([0-9\\.]+).*",
    },
    {
        "Name": "validation:loss",
        "Regex": ".*step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: ([0-9\\.]+) - val_accuracy: [0-9\\.]+.*",
    },
    {
        "Name": "sec/steps",
        "Regex": ".* (\d+)[mu]s/step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_accuracy: [0-9\\.]+",
    },
]
estimator = TensorFlow(
    entry_point='hld_sagemaker_demo.py',
    source_dir="/home/ec2-user/SageMaker/workshop_notebooks/chapter-3/src",
    role=ROLE_NAME,
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    py_version='py3',
    output_path=DESTINATION_BUCKET,
    image_uri='763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.4.1-gpu-py37-cu110-ubuntu18.04',
    metric_definition=keras_metric_definition,
    distribution={
        'parameter_server': {'enabled': True}
    }
)


## Train the model

Training the model is as simple as calling estimator.fit(), providing it appropriate arguments that is expected by `hld_sagemaker_demo.py`, which is the entry point for training the custom model (from the previous step)

In [ ]:
estimator.fit(
    {
        'train': train_images,
        'eval': val_images, 
        'test': test_images, 
        'log_dir': f'{DATA_FOLDER}/{LOG_FOLDER}/{log_file}'
    }, 
    wait=True
)

In [ ]:
estimator.deploy(initial_instance_count=1, instance_type='ml.t2.large')

# Deployment (move to chapter-4)

In [ ]:
from sagemaker.tensorflow import TensorFlowModel
model = TensorFlowModel(framework_version='2.4.1', model_data=f'{DESTINATION_BUCKET}/tensorflow-training-2021-05-05-10-10-34-979/output/model.tar.gz', role=ROLE_NAME)

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.large')


In [ ]:
predictor.predict(np.asarray([np.zeros((256, 256, 3))]))

In [ ]:
predictor

In [ ]:
model.endpoint_name

In [ ]:
model.predict